# Day 7: Handy Haversacks

[*Advent of Code 2020 day 7*](https://adventofcode.com/2020/day/7) and [*solution megathread*](https://redd.it/k8a31f)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2020/07/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2020%2F07%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

In [5]:

testdata = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.""".splitlines()

In [6]:
# testdata = '''ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
# iyr:2011 ecl:brn hgt:59in'''.splitlines()

inputdata = downloaded['input'].splitlines()

In [7]:
import re


def parseRules(data):
    pattern_re = r"""(?P<parent>.+)\sbags\scontain\s(?P<childList>.+)\."""
    pattern = re.compile(pattern_re, re.VERBOSE)
    childPattern_re = r"""(?P<count>\d+)\s(?P<color>[^,]+)\sbags*"""
    childPattern = re.compile(childPattern_re, re.VERBOSE)
    # light red bags contain 1 bright white bag, 2 muted yellow bags.
    output = dict()

    for line in data:
        # print(line)
        match = pattern.match(line)
        parent, childList = (match.group('parent'),
                             match.group('childList'))
        childMatches = re.findall(childPattern, childList)
        output[parent] = [(int(c[0]), c[1]) for c in childMatches]

    return output

In [8]:
def packingOptions(packThis, rules, debug: bool = False):
    options = set()
    for newColor in rules.keys():
        childList = rules[newColor]
        if packThis in [c for n, c in childList]:
            if debug:
                print(f"{newColor} can pack {packThis}" +
                      ", let us see which can pack that!")
            options.update([newColor])
            options.update(packingOptions(newColor, rules))
    return options

In [9]:
test_rules = parseRules(testdata)
options = packingOptions('shiny gold', test_rules, debug=True)
# print(options)
print(len(options))

bright white can pack shiny gold, let us see which can pack that!
muted yellow can pack shiny gold, let us see which can pack that!
4


In [10]:
rules = parseRules(inputdata)
options = packingOptions('shiny gold', rules)
# print(options)
print(len(options))

131


In [11]:
HTML(downloaded['part1_footer'])

## Part Two

In [12]:
HTML(downloaded['part2'])

In [13]:
def packBag(bag, rules):
    answer = 1
    if bag in rules.keys():
        for count, child in rules[bag]:
            answer += count * packBag(child, rules)
    return answer

In [14]:
answer = packBag('shiny gold', test_rules) - 1
print(answer)

32


In [15]:
answer = packBag('shiny gold', rules) - 1
print(answer)

11261


In [16]:
HTML(downloaded['part2_footer'])